In [21]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression

In [25]:
def wrangle(file):
    try:
        #read csv
        df = pd.read_csv(file)

        #remove whitespace in columns
        df.columns = df.columns.str.strip()

        # columns to drop
        cols_to_drop =['Adm5', 'Adm6', 'Geo1', 'econ4', 'acc_code', 'FndSrce1', 'function2', 'econ1', 'econ2', 'econ3', 'Year', 'Adm2', 'Adm3', 'Adm4']
        

        #mask = df['Class'].str.contains('Recurrent')
        #df = df[mask]

        

        #convert numerical columns to float
        num_cols = ['Estimates', 'Revised', 'Executed']
        for col in num_cols:
            df[col] = df[col].str.replace(',', '', regex=True).str.replace(r'[^\d\.\-]', '', regex=True).replace(['', '-'], np.nan).astype(float)
        
        #split funding source
        df[['acc_code', 'fund_source']] = df['FndSrce1'].str.lower().str.split(n=1, expand=True)

        #clean function 1
        df['function1'] = df['function1'].str.lower().str.strip()

        #drop columns
        df.drop(columns = cols_to_drop, inplace=True)
        
        #drop missing rows
        df.dropna(inplace=True)
        
        return df
        
    except Exception as e:
        print(f'Encountered {e} error')

    


In [26]:
df = wrangle('kenya-boost-2006-2018.csv')


/tmp/ipykernel_2760/2208220039.py:4: DtypeWarning: Columns (0,1,2,3,4,5,6,7,9,10,11,12,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


In [27]:
df['Econ0'].unique()

array(['Expenditures', 'Below the line', 'A-in-A'], dtype=object)

In [28]:
columns = df.columns
print(columns)
for col in columns:
    print(f'Value counts for {col} column')
    try:
        print(df[col].unique())
    except Exception as e:
        print(f"Error in column '{col}': {e}")
    print('------------------')
    print('******************')

Index(['Class', 'Adm1', 'Econ0', 'function1', 'Estimates', 'Revised',
       'Executed', 'fund_source'],
      dtype='object')
Value counts for Class column
['Development' 'Recurrent']
------------------
******************
Value counts for Adm1 column
['Central' 'Social security' 'Local Authorities' ' Central ']
------------------
******************
Value counts for Econ0 column
['Expenditures' 'Below the line' 'A-in-A']
------------------
******************
Value counts for function1 column
['agriculture and rural development' 'human resource development'
 'debt/pensions' 'governance, justice, law and order'
 'public administration and international relations'
 'environment, water and irrigation' 'trade, tourism and industry'
 'special programmes' 'physical infrastructure'
 'research,innovation and technology' 'national security' 'other']
------------------
******************
Value counts for Estimates column
[1.83186000e+07 1.27994500e+07 7.21840000e+06 ... 6.82732842e+10
 7.32605724

In [29]:
df.columns.str.strip()

Index(['Class', 'Adm1', 'Econ0', 'function1', 'Estimates', 'Revised',
       'Executed', 'fund_source'],
      dtype='object')

In [30]:
if 'Estimates' in df.columns:
    print(df['Estimates'].head())
else:
    print('Estimates column not there')

0    18318600.0
1    12799450.0
2     7218400.0
3    10025827.0
4      500000.0
Name: Estimates, dtype: float64


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77494 entries, 0 to 262022
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Class        77494 non-null  object 
 1   Adm1         77494 non-null  object 
 2   Econ0        77494 non-null  object 
 3   function1    77494 non-null  object 
 4   Estimates    77494 non-null  float64
 5   Revised      77494 non-null  float64
 6   Executed     77494 non-null  float64
 7   fund_source  77494 non-null  object 
dtypes: float64(3), object(5)
memory usage: 5.3+ MB


In [32]:
df['function1'] = df['function1'].str.lower().str.strip()


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77494 entries, 0 to 262022
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Class        77494 non-null  object 
 1   Adm1         77494 non-null  object 
 2   Econ0        77494 non-null  object 
 3   function1    77494 non-null  object 
 4   Estimates    77494 non-null  float64
 5   Revised      77494 non-null  float64
 6   Executed     77494 non-null  float64
 7   fund_source  77494 non-null  object 
dtypes: float64(3), object(5)
memory usage: 5.3+ MB
